In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import BingChatAccess
import ChatGPTAccess
import json
import os

In [3]:
chatGPT = ChatGPTAccess.ChatGPTAccess()

In [4]:
bingChat = BingChatAccess.BingChatAccess()

In [5]:
options = {"len" : 20, "adj" : "geographical", "top" : "Central asia", "foc" : "Focus on topics, which are important for the region", "pointsMin" : 4, "pointsMax" : 6}

In [6]:
with open('prompts.json') as f:
    prompts = json.load(f)

In [7]:
first = prompts["outline"].format(**options)

In [8]:
answer = chatGPT.answerPropt(first, 500, newConv=True)

Tokens: 106
ChatGPT: Using: gpt-3.5-turbo
ChatGPT: Sending prompt


In [9]:
topics  = json.loads(answer)
topics = topics[list(topics.keys())[0]]
print(topics)

['Introduction', 'Geography', 'Climate', 'Countries', 'Kazakhstan', 'Kyrgyzstan', 'Tajikistan', 'Turkmenistan', 'Uzbekistan', 'Historical Background', 'Silk Road', 'Nomadic Tribes', 'Cultural Diversity', 'Religions', 'Languages', 'Economy', 'Natural Resources', 'Agriculture', 'Tourism', 'Challenges and Opportunities']


In [10]:
outline = []
for top in topics:
    outline.append("    - " + top)
outline = "\n".join(outline)
print(outline)

    - Introduction
    - Geography
    - Climate
    - Countries
    - Kazakhstan
    - Kyrgyzstan
    - Tajikistan
    - Turkmenistan
    - Uzbekistan
    - Historical Background
    - Silk Road
    - Nomadic Tribes
    - Cultural Diversity
    - Religions
    - Languages
    - Economy
    - Natural Resources
    - Agriculture
    - Tourism
    - Challenges and Opportunities


In [11]:
with open('prompts.json') as f:
    prompts = json.load(f)

prompt = prompts["first_slide"].format(outline=outline, topOfSlide=topics[0], **options)

slides = []
slides.append(await bingChat.answerPropt(prompt, 1000, newConv=True))
print(slides[0]["text"])

Tokens: 307
BingChat: Using: gpt-4
BingChat: Sending prompt
BingChat: The try wasn't succesful, trying again
BingChat: Sending prompt
## Introduction
- Central Asia is a region of Asia that stretches from the **Caspian Sea** in the west to **western China and Mongolia** in the east, and from **Afghanistan and Iran** in the south to **Russia** in the north[^1^] [^2^].
- Central Asia consists of five former Soviet republics: **Kazakhstan, Kyrgyzstan, Tajikistan, Turkmenistan, and Uzbekistan**, which are colloquially referred to as the "-stans" [^1^].
- Central Asia has a varied geography, including high mountains (Tian Shan), vast deserts (Kyzyl Kum, Taklamakan), and especially treeless, grassy steppes [^2^].
- Central Asia has a dry climate, with hot summers and cool winters, and very little precipitation [^2^].
- Central Asia has a rich and diverse history and culture, influenced by its location along the **Silk Road**, the movement of nomadic tribes, and the interaction of different r

In [12]:
for i in range(1, len(topics)):
    gen = [slides[0]["text"]]
    for j in range(max(1, i-3), i-1):
        gen.append(slides[j]["text"])
    gen = "\n\n".join(gen)
    prompt = prompts["other_slide"].format(outline=outline, topOfSlide=topics[i], createdSlides=gen, **options)
    slides.append(await bingChat.answerPropt(prompt, 1000, newConv=True))
    print(slides[i]["text"])


Status code: 500
<!DOCTYPE html><html xml:lang="en" xmlns=http://www.w3.org/1999/xhtml xmlns:Web=http://schemas.live.com/Web/><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/><title>Bing</title><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/><meta name="viewport" content="width=device-width, initial-scale=1"></head><body><style>body,html{font:18px/normal "Segoe UI",Arial,Helvetica,Sans-Serif;line-height:23px;color:#333333;background:#D2D2D2 !important}a{color:#3366BB;text-decoration:none}#sw_content{width:750px;height:auto;margin:0 auto;padding:0;margin-bottom:150px;background:#D2D2D2}.panda{margin-top:60px}.panda img{width:350px;height:223px}.title{font-size:38px;line-height:48px;color:#333333;text-align:center;margin-top:30px;font-family:"Segoe UI",Arial,Helvetica,Sans-Serif}.sc_errD{font-size:13px;line-height:16px;color:#777777;text-align:center;margin-top:20px}.sc_error{margin-top:10px;padding:0 20px}.sc_error p{margin:18px 0px}.bing_logo{mar

In [14]:
pres = "\n\n".join(list(map(lambda x: x["text"], slides)))

name = "pres {}.md"
num = 1
while os.path.exists(name.format(num)):
    num+=1

with open(name.format(num), "w", encoding="UTF-8") as f:
    f.write(pres)